# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [ ]:
import pandas as pd #импортирую библиотеку Pandas и сохраняю в переменной pd
df = pd.read_csv() #открываю датафрейм data и сохраняю в переменную df
df.info() # вывожу на экран информацию о датафрейм

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

В датафрейм 21525 строк, 12 столбцов, типы данных по столбцам int64, float64, object. 
В столбцах 'days_employed' и 'total_income' есть пропуски данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [ ]:
df.isna().sum() #считаю пропуски в данных

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В столбцах 'days_employed' и 'total_income' пропущено 2174 значений, одинаковое количество. Пропуски формата 'Nan', т.к. тип данных в этих столбцах float64. Скорее всего ошибка оператора.
Так как пропущены данные о стаже работы и доходе клиента можно заполить пропуски медианными значениями групп клиентов по типу занятости.

Для заполнения пропусков в данных столбца 'days_employed' необходимо все занчения столбца привести к положительным числам, а также все значения записать в днях. Для этого сравню значения стажа с возрастом заемщиков. В столбце 'dob_years' также имеются недействительные значения возраста, заполню их медианными значениями групп заемщиков по типу занятости.

In [ ]:
for income in df['income_type'].unique(): # заменяем значения возраста 0 на медианные значения группы по типу занятости
    medians = df.loc[df['income_type'] == income,'dob_years'].median()#
    df.loc[(df['dob_years'] == 0.0) & (df['income_type'] == income),'dob_years']=medians#

Данные о детях, трудовом стаже и возрасте заемщиков должны быть представлены целым положительным числом, но в данных есть отрицательные значения, вернем их модуль.
Данные о трудовом стаже должны быть представлены в днях, данные о возрасте в годах. Проверив данные о трудовом стаже видим что есть слишком большие значения, сравним их с данными о возрасте клиентов. Для этого напишем цикл, который переберет все значения и индексы в столбце 'days_employed' и сравнит их со значениями столбца 'dob_years' сконвертированными в дни и часы.

In [ ]:
df['days_employed'] = abs(df['days_employed'])# возвращаю модуль значений столбца 'days_employed'
df['children'] = abs(df['children'])# возвращаю модуль значений столбца 'children'
df['total_income'] = abs(df['total_income'])# возвращаю модуль значений столбца 'total_income'

In [ ]:
for i, day in enumerate(df['days_employed']): # цикл пробегает по всем значениям и индексам значений в столбце 'days_employed'
    try: # в данных может встретиться недествительная информация, применим  try:-except для исключения ошибки кода
        if day <= (df.loc[i,'dob_years'] *365): #сравниваю значение в ячейке со значением ячейки возраста переведенным в дни
            # когда значение в столбце 'days_employed' меньше или равно значению столбца 'dob_years' оставляем значение столбца 
            # 'days_employed' без изменений.
            df.loc[i, 'days_employed'] = df.loc[i, 'days_employed'] #значение в столбце 'days_employed' не меняются
        elif day>(df.loc[i,'dob_years'] *365): #когда значение в столбце 'days_employed' больше значения столбца 'dob_years' 
            # меняем значение столбца 'days_employed' на значение в днях.
            df.loc[i, 'days_employed'] = df.loc[i, 'days_employed']/8760 
    except: # в случае недействительных данных код выведет предложение исправить данные и выведет строку с ошибкой.
        print('Исправьте данные.', df.loc[i]) 

In [ ]:
for income in df['income_type'].unique():     # создаю цикл, который перебирает значения типа занятости      
    medians = df.loc[df['income_type'] == income,'total_income'].median()# и вычисляет медианное значение в столбце 'total_income'
    df.loc[(df['total_income'].isna()) & (df['income_type'] == income),'total_income']=medians # и заполняет пропущенные значения медианными данными

In [ ]:
for incom in df['income_type'].unique():# этим циклом заполняю пропущенные значения стажа
    median = df.loc[df['income_type'] == incom,'days_employed'].median()# также медианными значениями 
    df.loc[(df['days_employed'].isna()) & (df['income_type'] == incom),'days_employed']=median #в группе по типу занятости

In [ ]:
df.info() #проверяю наличие пропусков

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


In [ ]:
df['education'] = df['education'].str.lower()# привожу данные в столбце об образовании в нижний регистр
df['family_status'] = df['family_status'].str.lower()# привожу данные в столбце об семейном положении в нижний регистр

In [ ]:
df['children'].unique() #проверяю коичество детей у заемщиков

array([ 1,  0,  3,  2,  4, 20,  5])

In [ ]:
child = (df.loc[df.loc[: ,'children'] == 20,'children'].count())/(df['children'].count()) # Вычисляю долю строк, содержащих
child*100 # 20 детей у заемщиков.

0.3530778164924506

В датафрем 76 строк имеют данные о наличии 20 детей у заемщиков даже возрастом 21 год, эти данные также ошибка ввода данных приписали лишний 0, удалим его.  
Доля всех строк, содержащих у заемщиков 20 детей составляет 0,35% от общего количества строк. Эти строки можно удалить без существенного влияния на конечный результат.

In [ ]:
df['children']=df['children'].replace(20,2) #меняю количество детей с 20 до 2

In [ ]:
df['children'].unique()# проверяем подготовку данных в столбце 'children'

array([1, 0, 3, 2, 4, 5])

**Вывод**

Пропущенные значения в данных о стаже и доходе заполнили медианными значениями в группах, сгруппированных по типу занятости. Провел работу с артефактами.


### Замена типа данных

In [ ]:
df['days_employed'] = df['days_employed'].astype('int') # меняем тип данных с 'float64' на 'int64' в столбце 'days_employed'


In [ ]:
df['total_income'] = df['total_income'].astype('int') # меняем тип данных с 'float64' на 'int64' в столбце 'total_income'

In [ ]:
df['dob_years'] = df['dob_years'].astype('int') # меняем тип данных с 'float64' на 'int64' в столбце 'total_income'

In [ ]:
df.info() # проверяем типы данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Типы данных в столбцах 'days_employed', 'dob_years' и 'total_income' изменились на 'int64'. Для изменения типов данных был применен метод  astype() так как он преобразует 'float64' на 'int64'. Данные типа 'float64' в столбце 'days_employed' появились в моиент перевода данных о стаже в дни или часы. 

### Обработка дубликатов

In [ ]:
df.duplicated().sum() # нахожу количество явных дубликатов
#df = df.drop_duplicates().reset_index(drop=True) #  Перенес эти строки ниже
#df.info()
dubl = (df.duplicated().sum())/(df['children'].count()) # Вычисляю долю строк содержащих дубликаты.
dubl*100 #

0.32984901277584205

Методом duplicated() обнаружили явные дубликаты, их доля составила 0,33% от общего количества строк. Удалим их.

In [ ]:
df = df.drop_duplicates().reset_index(drop=True) # удаляю строки с явными дубликатами
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [ ]:
#df['education'] = df['education'].str.lower()# привожу данные в столбце об образовании в нижний регистр
#df['family_status'] = df['family_status'].str.lower()# привожу данные в столбце об семейном положении в нижний регистр
df['gender'].unique() # проверяю уникальные значения в данных о поле клиента

array(['F', 'M', 'XNA'], dtype=object)

In [ ]:
df = df.drop(df[df['gender'] == 'XNA'].index)

In [ ]:
df['gender'].unique() # проверяю уникальные значения столбца 'gender'

array(['F', 'M'], dtype=object)

Пол клиента подразделен: 'F'- женщины, 'M'-мужчины, 'XNA'- ошибка в данных при заполнении анкеты, на результаты исследования не повлияет, поэтому удалим эту строку.

In [ ]:
df['income_type'].unique()#уникальные значения столбца тип занятости

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

Столбец 'income_type' неявных дубликатов не содержит.

**Вывод**

Для поиска явных дубликатов использовался метод duplicated(), он создан для автоматизации поиска дубликатов. Для удаления явных дубликатов использовался метод drop_duplicates() он удаляет строки вместе с индексами, для восстановления индексации использовался метод reset_index(drop=True). После удаления дубликатов количество строк уменьшилось до 21471, удаленные 54 строки никак не повлияют на результаты исследования. Столбцы 'education', 'family_status', 'income_type' неявных дубликатов не содержат, в столбце 'gender' в одной строке ошибка в данных при заполнении анкеты, на результат исследования не повлияет.   
Количество явных дубликатов составило 71, количество оставшихся строк 21454, достаточно для проведения анализа.

### Лемматизация

Для лемматизации выведем список уникальных значений целей кредитования и преобразуем их с тип str (список целей через запятую)

In [ ]:
purposes = list(df['purpose'].unique()) #возвращаю список уникальных значений в столбце 'purpose' и сохраняю в переменной 'purposes'
purposes =", ". join(purposes) # преобразую список 'purposes' в строку и сохраняю в переменную 'purposes'
display(purposes)# вывожу на дисплей

'покупка жилья, приобретение автомобиля, дополнительное образование, сыграть свадьбу, операции с жильем, образование, на проведение свадьбы, покупка жилья для семьи, покупка недвижимости, покупка коммерческой недвижимости, покупка жилой недвижимости, строительство собственной недвижимости, недвижимость, строительство недвижимости, на покупку подержанного автомобиля, на покупку своего автомобиля, операции с коммерческой недвижимостью, строительство жилой недвижимости, жилье, операции со своей недвижимостью, автомобили, заняться образованием, сделка с подержанным автомобилем, получение образования, автомобиль, свадьба, получение дополнительного образования, покупка своего жилья, операции с недвижимостью, получение высшего образования, свой автомобиль, сделка с автомобилем, профильное образование, высшее образование, покупка жилья для сдачи, на покупку автомобиля, ремонт жилью, заняться высшим образованием'

Импортируем библиотеку pymystem3 для лемматизации и библиотеку collections для подсчета лемматизированных слов.

In [ ]:
from pymystem3 import Mystem #импортирую библиотеку pymystem3 для лемматизации значений столбца 'purpose'
m = Mystem() # сохраняю библиотеку pymystem3 в переменную 'm'
from collections import Counter #импортирую библиотеку collections для подсчета слов 
lemmas = m.lemmatize(purposes) # лемматизирую уникальные значения столбца 'purpose'
Counter(lemmas) # подсчитываю количество слов 

Counter({'покупка': 10,
         ' ': 59,
         'жилье': 7,
         ', ': 37,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

**Вывод**

Наиболее часто встречающиеся существительные: 'автомобиль': 9 раз, 'образование': 9 раз,'недвижимость': 10 раз. На основании этих слов создадим категории целей кредитования: 'операции с недвижимостью', 'приобретение автомобиля', 'образование' и 'другое'.

### Категоризация данных

In [ ]:
# для категоризации создаю новый столбец 'purpose_group' в который функция purpose_group добавит значение категории
# на основании содержания контрольных слов в леммах значений столбца 'purpose'
def purpose_group(purpos):
    """Возвращает группу целей кредита, используя правило:
    - 'операции с недвижимостью', если в ячейке встречаются леммы 'недвижимость и жилье';
    - 'приобретение автомобиля', если в ячейке встречаются леммы 'автомобиль';
    - 'образование', если в ячейке встречаются леммы 'образование';
    - 'другое', в остальных случаях."""
    lemma = m.lemmatize(purpos)
    if 'недвижимость' in lemma:
        return 'операции с недвижимостью'
    if 'жилье' in lemma:
        return 'операции с недвижимостью'
    if 'автомобиль' in lemma:
        return 'приобретение автомобиля'
    if 'образование' in lemma:
        return 'образование'
    return 'другое'

df['purpose_group'] = df['purpose'].apply(purpose_group) # добавляю категорию целей кредитования на основании работы функции

In [ ]:
display(df['purpose_group'].value_counts()) # количество строк в категориях

операции с недвижимостью    10810
приобретение автомобиля      4306
образование                  4013
другое                       2324
Name: purpose_group, dtype: int64

Категоризируем по уровню дохода. Доходы до 250000 - 'низкие', от 250000 до 500000 - 'средние', выше 500000 - 'высокие'.

In [ ]:
def total_income_group(income):
    """Возвращает группу по доходу клиента, используя правило:
    - 'низкий', если доход составляет до 250000;
    - 'средний', если доход составляет от 250000 до 500000;
    - 'высокий', если доход составляет более 500000"""
    
    if income <250000:
        return 'низкий'
    if 250000 <income<500000:
        return 'средний'
    
    return 'высокий'

df['total_income_group'] = df['total_income'].apply(total_income_group)

In [ ]:
display(df['total_income_group'].value_counts()) # количество строк в категориях

низкий     18640
средний     2591
высокий      222
Name: total_income_group, dtype: int64

Категоризируем по наличию детей. Присвоим катагории 'Дети есть' и 'Детей нет'

In [ ]:
def children_group(child):
    """Возвращает группу по наличию детей у клиента:
    - 'Дети есть';
    - 'Детей нет'"""    
    if  child >0:
        return 'Дети есть'
    
    
    return 'Детей нет'

df['children_group'] = df['children'].apply(children_group)

In [ ]:
display(df['children_group'].value_counts()) # количество строк в категориях

Детей нет    14090
Дети есть     7363
Name: children_group, dtype: int64

**Вывод**

В выбранных категориях видно, что большая часть заявок на кредитование приходится на операции с недвижимостью, категория "другое" составила малую часть.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
report_child = df.groupby('children').agg({'debt':['count', 'sum', 'mean']})
report_child.columns = ['total', 'debt', '%']
report_child.sort_values(by='%', ascending=False)

,total,debt,%
children,,,
4,41,4,0.097561
2,2128,202,0.094925
1,4855,445,0.091658
3,330,27,0.081818
0,14090,1063,0.075444
5,9,0,0.000000


In [ ]:
report_child_group = df.groupby('children_group').agg({'debt':['count', 'sum', 'mean']})
report_child_group.columns = ['total', 'debt', '%']
report_child_group.sort_values(by='%', ascending=False)

,total,debt,%
children_group,,,
Дети есть,7363,678,0.092082
Детей нет,14090,1063,0.075444


**Вывод**

Из таблицы видно, что заемщики с детьми не вернули кредит в срок в 9,2 % случаев. Заемщики без детей не вернули кредит в срок в 7,54% случаев.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
report_family = df.groupby('family_status').agg({'debt':['count', 'sum', 'mean']})
report_family.columns = ['total', 'debt', '%']
report_family.sort_values(by='%', ascending=False)

,total,debt,%
family_status,,,
не женат / не замужем,2810,274,0.097509
гражданский брак,4150,388,0.093494
женат / замужем,12339,931,0.075452
в разводе,1195,85,0.071130
вдовец / вдова,959,63,0.065693


**Вывод**

Люди никогда не состоявшие в браке чаще не возвращают кредит в срок (9,75% случаев), чем люди в браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
report = df.groupby('income_type').agg({'total_income':['mean'], 'debt':['count', 'sum', 'mean']})
report.columns = ['total_income_sum', 'total', 'debt', '%']
report.sort_values(by='%', ascending=False)

,total_income_sum,total,debt,%
income_type,,,,
в декрете,53829.000000,1,1,1.000000
безработный,131339.000000,2,1,0.500000
сотрудник,159566.262901,11084,1061,0.095724
компаньон,199450.347843,5077,376,0.074059
госслужащий,168862.543583,1457,86,0.059025
пенсионер,135250.604597,3829,216,0.056412
предприниматель,499163.000000,2,0,0.000000
студент,98201.000000,1,0,0.000000


In [ ]:
report = df.groupby('income_type').agg({'total_income':['mean'], 'debt':['count', 'sum', 'mean']})
report.columns = ['total_income_sum', 'total', 'debt', '%']
report.sort_values(by='total_income_sum', ascending=False)

,total_income_sum,total,debt,%
income_type,,,,
предприниматель,499163.000000,2,0,0.000000
компаньон,199450.347843,5077,376,0.074059
госслужащий,168862.543583,1457,86,0.059025
сотрудник,159566.262901,11084,1061,0.095724
пенсионер,135250.604597,3829,216,0.056412
безработный,131339.000000,2,1,0.500000
студент,98201.000000,1,0,0.000000
в декрете,53829.000000,1,1,1.000000


In [ ]:
report = df.groupby('total_income_group').agg({'debt':['count', 'sum', 'mean']})
report.columns = ['total', 'debt', '%']
report.sort_values(by='%', ascending=False)

,total,debt,%
total_income_group,,,
низкий,18640,1547,0.082994
средний,2591,180,0.069471
высокий,222,14,0.063063


**Вывод**

Если судить по имеющимся данным, то заемщик в декрете точно не вернет кредит, но на основании этих данных нельзя делать такой вывод так как в исследование попал только один такой случай. На основании других, более полных данных видно, что заемщики с доходом чуть ниже медианного не вернули кредит в срок в 9,57% случаев.  При формировании таблицы по сгруппированным данным уровня дохода видно, что люди с низким доходом не вернули вовремя кредит в 8,3% случаях.

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
report_purpose = df.groupby('purpose_group').agg({'debt':['count', 'sum', 'mean']})
report_purpose.columns = ['total', 'debt', '%']
report_purpose.sort_values(by='%', ascending=False)

,total,debt,%
purpose_group,,,
приобретение автомобиля,4306,403,0.093590
образование,4013,370,0.092200
другое,2324,186,0.080034
операции с недвижимостью,10810,782,0.072340


**Вывод**

Из таблицы видно, что заемщики, потратившие средства на приобретение автомобиля не вернули кредит в срок в 9,35% случаев, когда заемщики средств на операции с недвижимость не вернули кредит в срок в 7,23% случаев.

## Шаг 4. Общий вывод

В ходе исследования надежности заемщиков проведена очистка статистики о платежеспособности клиентов банка от дубликатов, пропуски данных заполнены медианными значениями групп клиентов по типу их занятости. Все данные приведены к формату, облегчающему восприятие, обработку и анализ. Выделены группы клиентов по цели кредитования и сформированы выводы по каждому вопросу. Применена на практике лемматизация и котегоризация данных.</n>  
    
Определено, наличие детей почти на 2% увеличивают вероятность возврата кредита не в срок, люди никогда не состоявшие в браке или находящиеся в гражданском браке  не вернули кредит в срок на 2 % чаще чем люди когда-либо состаявшие в браке. 
    Низкий уровень дохода также увеличивает вероятность возврата кредита не в срок, но изменение порогов категорий доходов изменит и процент клиентов невовремя вернувших кредит. 
    Разницу в 2% между максимальными и минимальными долями в различных категориях считаю незначительной. 